In [5]:
import os 

In [6]:
%pwd

'd:\\Kidney-Disease-classification-cnn\\research'

In [7]:
os.chdir('../')

In [8]:
%pwd

'd:\\Kidney-Disease-classification-cnn'

In [17]:
#%pip install dagshub mlflow
import dagshub
dagshub.init(repo_owner='farhatsherani08', repo_name='Kidney-Disease-classification-cnn', mlflow=True)


Initialized MLflow to track repo "farhatsherani08/Kidney-Disease-classification-cnn"

Repository farhatsherani08/Kidney-Disease-classification-cnn initialized!

In [3]:
import tensorflow as tf

In [10]:
model=tf.keras.models.load_model('artifacts/training/trained_model.h5')


In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvalutionConfig:
    path_of_model: Path
    train_data_path: Path
    all_parameters: dict
    mlflow_url: str
    params_image_size:list
    params_batch_size:int

In [12]:
from src.cnn_classifier.constants import *
from src.cnn_classifier.utils.common import read_yaml, create_directories,save_json




In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_evaluation_config(self) -> EvalutionConfig:
         eval_config = EvalutionConfig(
            path_of_model="artifacts/training/trained_model.h5",
            train_data_path="artifacts/data_ingestion/kidney_dataset",
            mlflow_url="https://dagshub.com/farhatsherani08/Kidney-Disease-classification-cnn.mlflow",
            all_parameters=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
         return  eval_config

In [14]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evalution:
    def __init__(self, config: EvalutionConfig):
        self.config = config
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
       
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
       
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
       
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.train_data_path,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def _load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluate_model(self):
        self.model = self._load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        score = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=score)

    def log_into_mlflow(self):
        with mlflow.start_run():
            mlflow.log_params(self.config.all_parameters)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            
            mlflow.keras.log_model(
                self.model,
                artifact_path="model"
            )


In [16]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evalution = Evalution(eval_config)
    evalution.evaluate_model()
    evalution.log_into_mlflow()

except Exception as e:
    raise e

Found 2207 images belonging to 2 classes.


d:\Kidney-Disease-classification-cnn\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


69/69 ━━━━━━━━━━━━━━━━━━━━ 795s 12s/step - accuracy: 0.9021 - loss: 0.2488


2025/12/18 23:28:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/18 23:28:11 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


🏃 View run fearless-ant-880 at: https://dagshub.com/farhatsherani08/Kidney-Disease-classification-cnn.mlflow/#/experiments/0/runs/4fa98dabd5a5470f8a18092635b33cf6
🧪 View experiment at: https://dagshub.com/farhatsherani08/Kidney-Disease-classification-cnn.mlflow/#/experiments/0
